In [1]:
!pip install flask
!pip install pyngrok

In [2]:
!ngrok config add-authtoken 2hguiTCdH1helVkJDvY2c0DywcZ_Vc88G27KizP1sARLP2Gt

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok
from threading import Thread

app = Flask(__name__)


RATE_CARD = [
    (100, 4.71, 1.17),
    (300, 10.29, 1.17),
    (500, 14.55, 1.17),
    (float('inf'), 16.64, 1.17)
]

def calculate_electricity_cost(units):
    cost = 0
    remaining_units = units
    for bracket, rate, fixed_cost in RATE_CARD:
        if remaining_units <= 0:
            break
        if remaining_units > bracket:
            cost += bracket * rate
            remaining_units -= bracket
        else:
            cost += remaining_units * rate
            remaining_units = 0
        cost += fixed_cost
    return cost

@app.route('/', methods=['GET', 'POST'])
def index():
    results = None
    error = None
    try:
        if request.method == 'POST':
            state = request.form['state']
            monthly_bill = request.form.get('monthly_bill')
            units_consumption = request.form.get('units_consumption')
            roof_area_unit = request.form['roof_area_unit']
            roof_area = float(request.form['roof_area'])
            investment = float(request.form['investment'])
            required_capacity = float(request.form['required_capacity'])
            sanction_load = float(request.form['sanction_load'])
            residential = request.form.get('residential')

            if residential:

                sunlight_hours = 5
                panel_efficiency = 0.19
                emission_factor = 0.7


                if roof_area_unit == 'ft2':
                    roof_area = roof_area * 0.092903


                panel_capacity_from_area = roof_area * panel_efficiency * sunlight_hours

                if monthly_bill:

                    monthly_bill = float(monthly_bill)
                    monthly_kWh_consumption = 0
                    lower_bound = 0
                    for upper_bound, rate, fixed_cost in RATE_CARD:
                        if monthly_bill <= (upper_bound - lower_bound) * rate + fixed_cost:
                            monthly_kWh_consumption += (monthly_bill - fixed_cost) / rate
                            break
                        else:
                            monthly_kWh_consumption += (upper_bound - lower_bound)
                            monthly_bill -= (upper_bound - lower_bound) * rate + fixed_cost
                            lower_bound = upper_bound
                elif units_consumption:

                    units_consumption = float(units_consumption)
                    monthly_kwh_consumption = units_consumption
                    monthly_bill = calculate_electricity_cost(units_consumption)
                else:
                    error = "Please provide either the average monthly bill or average units consumption."
                    return render_template_string(template, results=results, error=error)


                panel_capacity_from_bill = monthly_kWh_consumption / (sunlight_hours * 30)


                recommendation = min(panel_capacity_from_area, panel_capacity_from_bill)


                yearly_energy = recommendation * 365 * sunlight_hours


                financial_savings = calculate_electricity_cost(yearly_energy)


                payback_period = investment / financial_savings


                roi = (financial_savings * 25 - investment) / investment * 100


                emission_savings = yearly_energy / 1000 * emission_factor * 25

                results = {
                    'energy': round(recommendation, 2),
                    'yearly_energy': round(yearly_energy, 2),
                    'financial_savings': round(financial_savings, 2),
                    'payback_period': round(payback_period, 2),
                    'roi': round(roi, 2),
                    'emission_savings': round(emission_savings, 2)
                }
            else:
                error = "Solar calculations are only for residential properties."

        return render_template_string(template, results=results, error=error)
    except Exception as e:
        return str(e)

template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Solar Rooftop Calculator</title>
    <link href="https://cdn.jsdelivr.net/npm/select2@4.1.0-rc.0/dist/css/select2.min.css" rel="stylesheet" />
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            background-color: #f9f9f9;
        }
        .container {
            background: #fff;
            padding: 30px;
            border-radius: 8px;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.1);
            max-width: 600px;
            width: 100%;
            text-align: center;
        }
        .form-group {
            margin-bottom: 20px;
            text-align: left;
        }
        label {
            display: block;
            margin-bottom: 5px;
            font-weight: bold;
        }
        input[type="text"],
        input[type="number"],
        select {
            width: calc(100% - 22px);
            padding: 10px;
            margin-bottom: 10px;
            border: 1px solid #ccc;
            border-radius: 4px;
        }
        button {
            padding: 12px 20px;
            background-color: #007bff;
            color: #fff;
            border: none;
            border-radius: 4px;
            cursor: pointer;
            font-size: 16px;
            transition: background-color 0.3s;
        }
        button:hover {
            background-color: #0056b3;
        }
        .error {
            color: red;
            margin-top: 10px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1 style="margin-bottom: 30px;">Solar Rooftop Calculator</h1>
        <form action="/" method="post" id="solarForm">
            <div class="form-group">
                <label for="state">Your State:</label>
                <select id="state" name="state" required>
                    <option value="" disabled selected>Select your state</option>
                    <option value="Andhra Pradesh">Andhra Pradesh</option>
                    <option value="Arunachal Pradesh">Arunachal Pradesh</option>
                    <option value="Assam">Assam</option>
                    <option value="Bihar">Bihar</option>
                    <option value="Chhattisgarh">Chhattisgarh</option>
                    <option value="Goa">Goa</option>
                    <option value="Gujarat">Gujarat</option>
                    <option value="Haryana">Haryana</option>
                    <option value="Himachal Pradesh">Himachal Pradesh</option>
                    <option value="Jharkhand">Jharkhand</option>
                    <option value="Karnataka">Karnataka</option>
                    <option value="Kerala">Kerala</option>
                    <option value="Madhya Pradesh">Madhya Pradesh</option>
                    <option value="Maharashtra">Maharashtra</option>
                    <option value="Manipur">Manipur</option>
                    <option value="Meghalaya">Meghalaya</option>
                    <option value="Mizoram">Mizoram</option>
                    <option value="Nagaland">Nagaland</option>
                    <option value="Odisha">Odisha</option>
                    <option value="Punjab">Punjab</option>
                    <option value="Rajasthan">Rajasthan</option>
                    <option value="Sikkim">Sikkim</option>
                    <option value="Tamil Nadu">Tamil Nadu</option>
                    <option value="Telangana">Telangana</option>
                    <option value="Tripura">Tripura</option>
                    <option value="Uttar Pradesh">Uttar Pradesh</option>
                    <option value="Uttarakhand">Uttarakhand</option>
                    <option value="West Bengal">West Bengal</option>
                </select>
            </div>
            <div class="form-group">
                <input type="checkbox" id="residential" name="residential">
                <label for="residential">Residential Property</label>
            </div>
            <div class="form-group">
                <label for="monthly_bill">Average Monthly Bill (Rs):</label>
                <input type="number" id="monthly_bill" name="monthly_bill">
            </div>
            <div class="form-group">
                <label for="units_consumption">Average Units Consumption (kWh):</label>
                <input type="number" id="units_consumption" name="units_consumption">
            </div>
            <div class="form-group">
                <div class="row">
                    <div class="col">
                        <label for="roof_area">Total Available Roof Top Area:</label>
                        <input type="number" id="roof_area" name="roof_area" required>
                    </div>
                    <div class="col">
                        <label for="roof_area_unit">Unit:</label>
                        <select id="roof_area_unit" name="roof_area_unit" required>
                            <option value="m2">m2</option>
                            <option value="ft2">ft2</option>
                        </select>
                    </div>
                </div>
            </div>
            <div class="form-group">
                <label for="investment">Amount available for investment:</label>
                <input type="number" id="investment" name="investment" required>
            </div>
            <div class="form-group">
                <label for="required_capacity">Required Solar Capacity (kW):</label>
                <input type="number" id="required_capacity" name="required_capacity" required>
            </div>
            <div class="form-group">
                <label for="sanction_load">Sanction Load (kW):</label>
                <input type="number" id="sanction_load" name="sanction_load" required>
            </div>
            <button type="submit">Calculate</button>
        </form>
        {% if results %}
            <div style="margin-top: 30px;">
                <h2>Results</h2>
                <ul style="list-style: none; padding: 0; text-align: left;">
                    <li>Recommended kW for Rooftop Installation: {{ results.energy }} kW</li>
                    <li>Yearly Energy Production: {{ results.yearly_energy }} kWh</li>
                    <li>Annual Financial Savings: Rs {{ results.financial_savings }}</li>
                    <li>Payback Period: {{ results.payback_period }} years</li>
                    <li>Return on Investment (ROI): {{ results.roi }}%</li>
                    <li>25-year Emission Savings: {{ results.emission_savings }} tonnes of CO2</li>
                </ul>
            </div>
        {% endif %}
        {% if error %}
            <p class="error">{{ error }}</p>
        {% endif %}
    </div>
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/select2@4.1.0-rc.0/dist/js/select2.min.js"></script>
    <script>
        $(document).ready(function() {
            $('#state').select2();
        });
    </script>
</body>
</html>
'''

if __name__ == '__main__':

    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")


    thread = Thread(target=app.run, kwargs={"use_reloader": False})
    thread.start()



 * ngrok tunnel "NgrokTunnel: "https://cb0e-34-86-88-153.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
